In [ ]:
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import CommandJob, Code, PipelineJob, ResourceConfiguration

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

In [ ]:
#define the job to get data
get_data_cmd = 'bash main.sh ${{outputs.cifar}}'
get_data_outputs = {"cifar":None}
get_data_job = CommandJob(
    command = get_data_cmd,
    outputs = get_data_outputs,
    environment = 'AzureML-Minimal:18',
    compute = 'cpu-cluster',
    code = Code(local_path="./get-data")
)

In [ ]:
#define the job to train the model
train_model_cmd = 'python main.py --data-dir ${{inputs.cifar}} --epochs ${{inputs.epochs}} --model-dir ${{outputs.model_dir}}'
train_model_inputs = {'epochs': 1, 'cifar' : '${{jobs.get-data.outputs.cifar}}'}
train_model_outputs = {'model_dir': None}
train_model_job = CommandJob(
    command = train_model_cmd,
    inputs = train_model_inputs,
    outputs=train_model_outputs,
    code = Code(local_path="./train-model"),
    distribution = {
        'distribution_type': 'PyTorch',
        'process_count_per_instance': 2},
    resources=ResourceConfiguration(instance_count=2),
    environment = 'AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu:6',
    compute = 'gpu-cluster'
)

In [ ]:
#define the job to evaluate the model
eval_model_cmd = 'python main.py --data-dir ${{inputs.cifar}} --model-dir ${{inputs.model_dir}}/model'
eval_model_inputs = {'cifar': '${{jobs.get-data.outputs.cifar}}', 'model_dir': '${{jobs.train-model.outputs.model_dir}}'}
eval_model_job = CommandJob(
    command = eval_model_cmd,
    inputs = eval_model_inputs,    
    code = Code(local_path="./eval-model"),
    distribution = {
        'distribution_type': 'PyTorch',
        'process_count_per_instance': 2},
     resources=ResourceConfiguration(instance_count=2),
    environment = 'AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu:6',
    compute = 'gpu-cluster'
)

In [ ]:
pipeline_job = PipelineJob(
    description = 'cifar-10-pipeline-example',
    jobs = {'get-data': get_data_job, 'train-model': train_model_job, 'eval-model': eval_model_job}
)

In [ ]:
#submit the pipeline job
returned_job = ml_client.jobs.create_or_update(pipeline_job)
#get a URL for the status of the job
returned_job.services["Studio"].endpoint